Import

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

Load

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Read

In [ ]:
long_df = pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")
long_df

Reduce df

In [ ]:
df = long_df[: 1000]

In [ ]:
df.info()

Check for null values

In [ ]:
df.isnull().sum()

Analyse prices

In [ ]:
sns.displot(df['price'])

Plot map of houses

Anayse independent features

In [ ]:
bedrooms = df['bedrooms'].value_counts().to_frame().sort_index()     
bedrooms

In [ ]:
bathrooms = df['bathrooms'].value_counts().to_frame().sort_index()
bathrooms

In [ ]:
floors = df['floors'].value_counts().to_frame().sort_index()
floors

In [ ]:
sns.boxplot(x='bedrooms', y='price', data=df)

In [ ]:
sns.boxplot(x='condition', y='price', data=df)

In [ ]:
y = df['price']
x = df['sqft_living']
plt.scatter(x,y)

plt.title('Scatterplot living space vs price')
plt.xlabel('living space')
plt.ylabel('price')

In [ ]:
y = df['price']
x = df['sqft_lot']
plt.scatter(x,y)

plt.title('Scatterplot lot space vs price')
plt.xlabel('living space')
plt.ylabel('price')

In [ ]:
plt.figure(figsize=(10,10))
sns.jointplot(x=df.lat.values, y=df.long.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()
sns.despine

Drop id, price and date

In [ ]:
target = df['price']

df.drop(['id', 'date', 'price'], axis=1, inplace=True)
df

Heatmap

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

Drop low correlation columns

In [ ]:
df.drop(['yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'], axis=1, inplace=True)
df

Define X and y

In [ ]:
y = target
X = df

Standardise

In [ ]:
X = (X - np.average(X)) / (np.std(X))
X

Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1)
X_train.shape, X_val.shape, y_train.shape,y_val.shape

Select Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor

ET = ExtraTreesRegressor()

parameters = {'n_estimators':[100, 300, 500], 'criterion':('mse', 'mae'), 'max_features': ('auto', 'sqrt', 'log2')}

reg = GridSearchCV(ET, parameters).fit(X_train, y_train)
print(reg.best_params_)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor(criterion='mae', max_features='log2', n_estimators=300,random_state=1).fit(X_train, y_train)
X_pred = (model.predict(X_train))
print(model.score(X_train, y_train))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_train, X_pred, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

In [ ]:
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})
df

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_val, y_pred, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()